<table style="width:100%; background-color: #EBF5FB">
  <tr>
    <td style="border: 1px solid #CFCFCF">
      <b>Household data: Processing Notebook</b>
      <ul>
        <li><a href="main.ipynb">Main Notebook</a></li>
        <li>Processing Notebook</li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/household_data">Household Data Package</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
    </td>
  </tr>
</table>

# Table of Contents
* [1. Introductory Notes](#1.-Introductory-Notes)
* [2. Settings](#2.-Settings)
	* [2.1 Import Python libraries](#2.1-Import-Python-libraries)
	* [2.2 Set version number and recent changes](#2.2-Set-version-number-and-recent-changes)
	* [2.3 Select timerange](#2.3-Select-timerange)
	* [2.4 Select download source](#2.4-Select-download-source)
* [3. Download](#3.-Download)
* [4. Read](#4.-Read)
	* [4.1 Preparations](#4.1-Preparations)
	* [4.2 Select household subset](#4.2-Select-household-subset)
	* [4.3 Reading loop](#4.3-Reading-loop)
	* [4.4 Save raw data](#4.4-Save-raw-data)
* [5. Processing](#5.-Processing)
	* [5.1 Validate Data](#5.1-Validate-Data)
    * [5.2 Aggregate Index equidistantly](#5.2-Aggregate-Index-equidistantly)
    * [5.3 Missing Data Handling](#5.3-Missing-Data-Handling)
    * [5.4 Aggregate hourly and 15min interval data](#5.4-Aggregate-hourly-and-15min-interval-data)
    * [5.5 Insert a column with Central European Time](#5.5-Insert-a-column-with-Central-European-Time)

# 1. Introductory Notes

This Notebook handles missing data, performs calculations and aggragations and creates the output files.

# 2. Settings

## 2.1 Import Python libraries

This section: load libraries and set up a log.

In [ ]:
# Python modules
from datetime import datetime, date, timedelta, time
import pandas as pd
import numpy as np
import logging
import json
import sqlite3
import yaml
import itertools
import os
import pytz
import hashlib
from shutil import copyfile

# Scripts from household repository package
from household.download import download
from household.read import read
from household.validation import validate
from household.imputation import make_equidistant
from household.imputation import find_nan
from household.imputation import resample_markers

# Reload modules with execution of any code, to avoid having to restart
# the kernel after editing timeseries_scripts
%load_ext autoreload
%autoreload 2

households_yaml_path = 'input/households.yml'

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('log')
# For more detailed logging messages, replace 'INFO' with 'DEBUG'
# (May slow down computation).
#logger.setLevel('DEBUG')

## 2.2 Set version number and recent changes

Executing this script till the end will create a new version of the data package.
The Version number specifies the local directory for the data <br>
We include a note on what has been changed.

In [ ]:
version = '2017-06-30'
data_path = os.path.join(version, 'original_data')
changes = '''Initial upload'''

## 2.3 Select timerange

Select the time range to read and process data. <br>
*Default: all data.*

Type `None` to process all available data.

In [ ]:
start_from_user = None  # i.e. date(2016, 1, 1)
end_from_user = None  # i.e. date(2016, 1, 31)

## 2.4 Select download source

The raw data can be downloaded as a zip file from the OPSD Server. To do this, specify an archive version to use, that has been cached on the OPSD server as input.

In [ ]:
archive_version = '2017-06-30' # i.e. '2017-06-30'

# 3. Download

This section: download raw data to process.

If the original data does not exist, it will be downloaded from the OPSD Server and extracted in a local directory

In [ ]:
download(version=archive_version)

# 4. Read

This section: Read each downloaded file into a pandas-DataFrame and merge data from different sources if it has the same time resolution.

## 4.1 Preparations

Set the title of the rows at the top of the data used to store metadata internally. The order of this list determines the order of the levels in the resulting output.

In [ ]:
headers = ['project', 'region', 'household', 'type', 'feed']

Create an empty dictionary to be populated by the read function

In [ ]:
data_households = {}

## 4.2 Select household subset

Optionally, specify a subset of households to process. <br>
The next cell prints the available sources and datasets.

In [ ]:
with open(households_yaml_path, 'r') as f:
    households = yaml.load(f.read())
for k, v in households.items():
    print(yaml.dump({k: list(v['feeds'].keys())}, default_flow_style=False))

Copy from its output and paste to following cell to get the right format.

Type `subset = None` to include all data.

In [ ]:
subset = yaml.load('''
    insert_household_here:
    - insert_feed1_here
    - insert_feed2_here
    more_households_here:
    - more_feeds_here
    ''')

subset = None

Now eliminate households and feeds not in subset.

In [ ]:
if subset:  # eliminate households and feeds not in subset
    households = {household_name: {k: v
                                   for k, v in households[household_name].items()
                                   for k, v in households[household_name]['feeds'].items()
                                   if k in feed_list}
                  for household_name, feed_list in subset.items()}

## 4.3 Reading loop

Loop through households and feeds to do the reading

In [ ]:
# For each source in the household dictionary
for household_name, household_dict in households.items():
    df = read(household_name, household_dict['dir'], household_dict['feeds'], 
              household_dict['project'], household_dict['region'], household_dict['type'], headers, 
              start_from_user=start_from_user,
              end_from_user=end_from_user)
    
    data_households[household_name] = df

## 4.4 Save raw data

Save the DataFrames created by the read function to disk. This way you have the raw data to fall back to if something goes wrong in the remainder of this notebook without having to repeat the previos steps.

In [ ]:
for household_name, household_dict in households.items():
    data_households[household_name].to_pickle('raw_'+household_dict['dir']+'.pickle')

Load the DataFrames saved above

In [ ]:
data_households = {}
for household_name, household_dict in households.items():
    data_households[household_name] = pd.read_pickle('raw_'+household_dict['dir']+'.pickle')

# 5. Processing

This section: validate energy data and fix wrongly measured data points. Handle missing data and aggregation to regular and 15 minute intervals.

## 5.1 Validate Data

With the raw data being the unvalidated measurements of a research prototype under development, certain steps should be taken, to remove clearly incorrect measured data points or react to other events, such as the counter reset of an energy meter.

In [ ]:
for household_name, household_dict in households.items():
    data_households[household_name] = validate(data_households[household_name], household_name, 
                                               household_dict['feeds'], household_dict['adjustments'], 
                                               headers)

## 5.2 Aggregate Index equidistantly

Most of the acquired household data comes in irregular time intervals and is highly unsynchronized for different data feeds. 

To improve readability and comparability, regular intervals will be assumed and interpolated according to existing values. Gaps greater than 15 minutes will be left untouched.

In [ ]:
data_sets = {}
for household_name, household_dict in households.items():
    for res_dict in household_dict['resolutions']:
        res_key = res_dict['key']
        
        df = make_equidistant(data_households[household_name], household_name, 
                              res_key, res_dict.get('seconds'), 
                              res_dict.get('start'), res_dict.get('end'), 
                              household_dict['feeds'])
        
        if not res_key in data_sets:
            data_sets[res_key] = df
        elif not df.empty:
            data_sets[res_key] = (
                data_sets[res_key]
            ).combine_first(df)

Save/Load the equidistant data sets

In [ ]:
for res_key, df in data_sets.items():
    df.to_pickle('fixed_'+res_key+'.pickle')

In [ ]:
data_sets = {}
data_sets['1min'] = pd.read_pickle('fixed_1min.pickle')
data_sets['3min'] = pd.read_pickle('fixed_3min.pickle')

## 5.3 Missing Data Handling

Patch missing data. At this stage, only small gaps (up to 2 hours) are filled by linear interpolation. This catched most of the missing data due to daylight savings time transitions or failed radio transmissions, while leaving bigger gaps untouched.

The exact locations of missing data are stored in the `nan_table` DataFrames.

Where data has been interpolated, it is marked in a new column `comment`. For eaxample the comment `residential_004_pv;` means that in the original data, there is a gap in the solar generation timeseries from the Resident 4 in the time period where the marker appears.

Patch the datasets and display the location of missing Data in the original data.

In [ ]:
nan_sets = {}
for res_key, df in data_sets.items():
    data_sets[res_key], nan_table = find_nan(df, headers, res_key, patch=True)
    
    nan_sets[res_key] = nan_table

Execute this to see an example of where the data has been patched.

In [ ]:
data_sets['3min'][data_sets['3min']['interpolated_values'].notnull()].tail()

In [ ]:
data_sets['1min'][data_sets['1min']['interpolated_values'].notnull()].tail()

Display the table of regions of missing values

In [ ]:
nan_sets

You can export the NaN-tables to Excel in order to inspect where there are NaNs

In [ ]:
writer = pd.ExcelWriter('NaN_table.xlsx')
for res_key, df in nan_sets.items():
    df.to_excel(writer, res_key)
writer.save()

Save/Load the patched data sets

In [ ]:
for res_key, df in data_sets.items():
    df.to_pickle('patched_'+res_key+'.pickle')

In [ ]:
data_sets = {}
data_sets['1min'] = pd.read_pickle('patched_1min.pickle')
data_sets['3min'] = pd.read_pickle('patched_3min.pickle')

## 5.4 Aggregate hourly and 15min interval data

As some data comes in 1-minute, other (older series) in 3-minute intervals, a harmonization can be done. With most billing intervals being at 15-minute and 60-minute ranges, a resampling to those intervals can be done to improve the overview over the data.

The marker column is resampled separately in such a way that all information on where data has been interpolated is preserved.

To do this, first combine the patched resolutions again

In [ ]:
data_full = None
for res_key, df in data_sets.items():
    if data_full is None:
        data_full = df
    elif not df.empty:
        data_full = data_full.combine_first(df)

Afterwards, condense the marker column to both 15 and 60 minutes resolution, resample the series and update the condensed markers

In [ ]:
start_15 = data_full.index[0].replace(minute=data_full.index[0].minute + (15 - data_full.index[0].minute % 15), second=0)
end_15 = data_full.index[-1].replace(minute=data_full.index[-1].minute - (data_full.index[-1].minute % 15), second=0)
index_15 = pd.DatetimeIndex(start=start_15, end=end_15, freq='15min')
marker_15 = data_full['interpolated_values'].groupby(
    pd.Grouper(freq='15min', closed='left', label='left')
    ).agg(resample_markers).reindex(index_15)

data_sets['15min'] = data_full.resample('15min').last()
data_sets['15min']['interpolated_values'] = marker_15

In [ ]:
start_60 = data_full.index[0].replace(minute=0, second=0)
end_60 = data_full.index[-1].replace(minute=0, second=0)
index_60 = pd.DatetimeIndex(start=start_60, end=end_60, freq='60min')
marker_60 = data_full['interpolated_values'].groupby(
    pd.Grouper(freq='60min', closed='left', label='left')
    ).agg(resample_markers).reindex(index_60)

data_sets['60min'] = data_full.resample('60min').last()
data_sets['60min']['interpolated_values'] = marker_60

## 5.5 Insert a column with Central European Time

The index column of th data sets defines the start of the timeperiod represented by each row of that data set in **UTC** time. We include an additional column for the **CE(S)T** Central European (Summer-) Time, as this might help aligning the output data with other data sources.

In [ ]:
info_cols = {'utc': 'utc_timestamp',
             'cet': 'cet_cest_timestamp',
             'marker': 'interpolated_values'}

In [ ]:
for res_key, df in data_sets.items():
    if df.empty:
        continue
    df.index.rename(info_cols['utc'], inplace=True)
    df.insert(0, info_cols['cet'],
              df.index.tz_localize('UTC').tz_convert('Europe/Brussels'))

Create a final savepoint

In [ ]:
for res_key, df in data_sets.items():
    df.to_pickle('final_'+res_key+'.pickle')

In [ ]:
data_sets = {}
data_sets['1min'] = pd.read_pickle('final_1min.pickle')
data_sets['3min'] = pd.read_pickle('final_3min.pickle')
data_sets['15min'] = pd.read_pickle('final_15min.pickle')
data_sets['60min'] = pd.read_pickle('final_60min.pickle')